In [1]:
import cv2
import re
import numpy as np
import pandas as pd
import seaborn as sns
import collections
import sys
import time
import networkx as nx
from functools import reduce
from pyvis.network import Network
from functools import reduce
from networkx.algorithms.dag import transitive_reduction
sys.path.append("/booleanfs2/sahoo/Hegemon/")
sys.path = ["/booleanfs2/sahoo/BoNE/"] + sys.path
import HegemonUtil as hu

try:
    reload  # Python 2.7
except NameError:
    try:
        from importlib import reload  # Python 3.4+
    except ImportError:
        from imp import reload  # Python 3.0 - 3.3

import BNETUtils as BU
import bone
reload(BU)
reload(hu)
reload(bone)

<module 'bone' from '/booleanfs2/sahoo/BoNE/bone.py'>

In [2]:
# define three groups of functional, dysfunctional and shared   
fn_n=['CLU','SPP1','UCMA','GDF5','CLEC3A','CA2','HP','DUSP2','ZNF385B','MT1G','FBXO2','EFHD1','AURKB','UCP2','H2AC14','H2AC17','TENT5B','NIM1K','DLK1','GRP','CCN6','CNMD','BTC','IHH','WIF1','SFRP5','IBSP','MYBPC2','AGTR2','KCTD4','OSR2','SOX9','AGR2','NOTUM','WNK4','KIF1A','LIX1','RAPGEF4','COMP','BSN','COL2A1','MMP13','COL11A1','ACAN','COL9A1','FMOD','CADM2','MATN3','MPZL2','VEGFA','CDH15','FN1','SERPINA1','ITIH6','RAC2','SNORA7A','S100P','TINAGL1','TGFA','ADRB2','CDC45','MCOLN2','SNORA8','FERMT1','CRYBA2']+['AGR2', 'CCDC26', 'SLCO2B1', 'ACRBP', 'MYL4', 'PHOSPHO1', 'H1-4']
dysf_n=['DDIT4','CALM2','CTHRC1','LAPTM4A','MYL6','ANXA2','B2M','LGALS1','CD86','ACTB','COL1A1','CD99','CORT','SLITRK6','METTL11B','LHX8','COL6A1','IFITM3','ANXA5','CCN2','ITM2B','CXCL14','SLIT3','MDK','SERPINF1','SFRP2','RARRES2','PTN','SFRP4','THY1','PDGFRB','RUNX2','PRRX1','JUNB','CDH5','DPEP1','ALPL','ISG15','CTSK','FTL','MMP2','ACTA2','LUM','COL3A1','COL6A3','PCOLCE','COL1A2','MFAP2','MXRA8','BGN','FBLN1','ASPN','POSTN','TAGLN','DCN','IL11RA','HEPN1','WNT5A','CCN5','IL34','BMPR1A','ACVRL1','TGFB3']+['MRFAP1', 'PRSS23', 'MYL9', 'TNFRSF14', 'RILP', 'ANXA5', 'NUPR1', 'ZNF106']
sh_n=['LIF','MSLN','IL7','CXCL12','GLI3','SMO','IGF2','IGF1','FGF7','ANGPT1','GRB2','EGFR','CSF1R','CD47','BMPR1B','NOTCH3','LRP5','KDR','ATF2','RUNX1','PRRX2','FBXO7','CPEB1','FAM104A','RRBP1','TENT2','FTH1','PRELP','ACTA1','GLI1','GLI2','TNXA','CLDN5','IL11','CCL19','LBP']+ ['RIPK3', 'HECTD2', 'BACE1', 'ANGPT1', 'DIP2C', 'ITSN1', 'OSMR', 'B3GNT8']


In [3]:
# Find the best probe ids for genes whose symbols are found in the global dataset (GL1) and store unavailable genes in BU.NAG
db = hu.Database("/booleanfs2/sahoo/Hegemon/explore.conf")
h = hu.Hegemon(db.getDataset("GL1"))
h.init()
h.initPlatform()



BU.NAG=[]

BP_available=BU.BESTID(fn_n+sh_n+dysf_n) # Find the best probeId for each gene

In [4]:
# Check which aliases are available in the dataset for each unavaiable gene, and update each group with the best probeid for each alias 
Alias_NAG=BU.add_aliases_set(BU.NAG)

update_nag=[]
updated_fn=[]
updated_sh=[]
updated_dys=[]
for k in Alias_NAG:
    for i in Alias_NAG[k]:
        j=h.getBestID(h.getIDs(i).keys())
        if j is not None:
            update_nag.append((k,i))
            if k in dysf_n:
                updated_dys.append(i)
            elif k in fn_n:
                updated_fn.append(i)
            else:
                updated_sh.append(i)
                
# Get the unavailable genes' best IDs                
BP_unavailable=BU.BESTID(updated_sh+updated_fn+updated_dys)

fn_n=fn_n+updated_fn
dysf_n=dysf_n+updated_dys
sh_n=sh_n+updated_sh


BP_available.update(BP_unavailable)


querying 1-19...done.
Finished.
1 input query terms found no hit:
	['METTL11B']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [5]:
# Boolean relationships between genes are stored as matrices based on the two different Boolean thresholds (p(error rate = 0.1 or 0.15))
# Each element of matrxi is 0 (NO Boolean relationships) or a number associated with THE six different types of Boolean relationships between pairs of genes:
# 1: LOW-HIGH (lohi), 
# 2: LOW-LOW (lolo)
# 3: HIGH-HIGH (hihi)   
# 4: HIGH-LOW (hilo)
# 5: Equivalent (eqv)
# 6: Opposite (opo)


cfile = "Global-net-3-0.15.rl"
net = bone.BINetwork(cfile)
net.init()


balanced_ids = {k: v for k,v in BP_available.items() if v in list(net.balanced)}
balanced_ids,balanced_ids
tbl_h15=BU.relmatrix(balanced_ids,balanced_ids,net,"BestID_3-0.15_relmatrix.txt")

#---------------
cfile = "Global-net-3-0.1.rl"
net = bone.BINetwork(cfile)
net.init()

balanced_ids = {k: v for k,v in BP_available.items() if v in list(net.balanced)}
balanced_ids,balanced_ids
tbl_h1=BU.relmatrix(balanced_ids,balanced_ids,net,"BestID_3-0.10_relmatrix.txt")


In [6]:

presenting_probes=list(tbl_h1.columns) # presenting_probes=presenting_genes+"#"+last three digits of probeID
presenting_genes=set(map(lambda i:i.split('#')[0], presenting_probes)) #  gene symbols
dysf = [i for i in presenting_probes if i.split('#')[0] in dysf_n and i.split('#')[0] not in sh_n ] #dys= genes associated to dysfunctional
fn = [i for i in presenting_probes if i.split('#')[0] in fn_n and i.split('#')[0] not in sh_n ] # fn= genes associated to functional 
shared = [i for i in presenting_probes if i.split('#')[0] in sh_n ] # Neither functional nor dysfunctional 



nodes = set()
l = len(tbl_h15)
edges = []

for i in range(0, l):
    for j in range(i + 1, l):
            if (int(tbl_h1.iloc[i,j]) == 5) and (int(tbl_h15.iloc[i,j]) == 5): #  When an element in both matrices is 5 at the same time, two genes have equivalent relations
                list_edge_h = int(tbl_h1.iloc[i,j])
            elif int(tbl_h15.iloc[i,j]) != 5: 
                list_edge_h = int(tbl_h15.iloc[i,j])
            else:
                list_edge_h = int(tbl_h1.iloc[i,j])
            if list_edge_h != 0:
                edges.append((tbl_h15.index[j], tbl_h15.index[i], str(list_edge_h))) # x,y,Boolean relationships between (x,y)
                nodes.add(tbl_h15.index[j]) 
                nodes.add(tbl_h15.index[i])

len(edges)

5352

In [7]:
# Create an object from the class of Gene, defined in BNETUtils.py

genes = BU.create(edges, fn, dysf)  



In [8]:
# The genes have equivalent relationships to any genes from the shared group, clustered as a new node, and relabelled as a new shared gene
genes_BU=genes
df_beforeClustering=dysf
fn_beforeClustering=fn
res_sh_dys, res_sh_fn,res_sh_dys_fn,res_dys_fn = BU.find_final_clusters(genes)




###

###
moved_and_shared_genes = reduce(lambda x, y: x + y, res_sh_dys_fn,[])
dysf = [x for x in dysf if x not in moved_and_shared_genes]  # remove the new shared genes from the dysfunctional list
fn =  [x for x in fn if x not in moved_and_shared_genes] # remove the new shared genes from the functional list


edges_new,genes_cluster=BU.Update_edges_after_clustering(moved_and_shared_genes,res_sh_dys_fn, edges) # update the edges or relationships between genes


(ZNF385B#019, func)
(CLU#792, func)
(FBXO2#305, func)
(EFHD1#343, func)
(IHH#358, func)
(WIF1#712, func)
(MYBPC2#394, func)
(KCTD4#787, func)
(KIF1A#482, func)
(LIX1#865, func)
(RAPGEF4#651, func)
(BSN#586, func)
(COL2A1#492, func)
(CADM2#754, func)
(CCDC26#849, func)
(MYL4#395, func)
(PHOSPHO1#218, func)
(PRELP#223, shared)
(TNXA#093, shared)
(CLDN5#482, shared)
(LBP#461, shared)
(CALM2#243, dysfunc)
(ACTB#594, dysfunc)
(SLIT3#813, dysfunc)
(PTN#466, dysfunc)
(DPEP1#983, dysfunc)
(FTL#788, dysfunc)
(HIST1H1E#553, func)
(WISP2#792, dysfunc)
(ZNF474#907, dysfunc)
(GRP#326, func)
(SPP1#875, func)
(COL11A1#892, func)
(VEGFA#512, func)
(ACRBP#717, func)
(ANGPT1#939, shared)
(FBXO7#178, shared)
(FAM104A#319, shared)
(ACTA1#872, shared)
(B3GNT8#338, shared)
(COL6A1#428, dysfunc)
(ANXA5#782, dysfunc)
(ITM2B#732, dysfunc)
(HIST1H2AJ#583, func)
(HIST1H2AM#583, func)
(HP#697, func)
(CLEC3A#802, func)
(DUSP2#794, func)
(AURKB#464, func)
(DLK1#560, func)
(BTC#412, func)
(FN1#199, func)
(MCOLN2#110

In [9]:
genes_cluster

defaultdict(None,
            {'SFRP5#468': 'C16orf77#046_CDH15#328_IBSP#370_SFRP5#468',
             'IBSP#370': 'C16orf77#046_CDH15#328_IBSP#370_SFRP5#468',
             'C16orf77#046': 'C16orf77#046_CDH15#328_IBSP#370_SFRP5#468',
             'CDH15#328': 'C16orf77#046_CDH15#328_IBSP#370_SFRP5#468',
             'KDR#934': 'CDH5#677_KDR#934',
             'CDH5#677': 'CDH5#677_KDR#934'})

In [10]:
# update the genes with new rels
genes = BU.create(edges_new, fn, dysf)  


In [11]:
#1-find all chains of lolo pathways starting from shared or dysfunctional lists and ending with a gene from the dysfunctional list


    
BU.saved_dys =[]
BU.saved_shared =[]
BU.edge_write = []
BU.nodes_in_path = []

start = time.perf_counter()
for g in dysf:
    print(g)
    g1 = genes[g]
    g1.dfs('2', 'dysfunc', 1, [g1], [], lambda p, g: BU.lolo_path([], p,g, t_m='hilo'),'lolo')        
        
# print('all low-low pathways start with dysfunc list done')        
for gg in genes.keys():
    g1 = genes[gg]
    if g1.ct != 'func' and g1.ct!='dysfunc' : # start from the shared list
        print(g1.pretty())
        g1.dfs('2', 'shared', 1, [g1], [], lambda p, g: BU.lolo_path([], p, g, t_m='hilo_shared'),'lolo')
        g1.dfs('2', 'dysfunc', 1, [g1], [], lambda p, g: BU.lolo_path([], p, g, t_m='hilo'),'lolo')
# print('all low-low pathways start with shared list and end with dysfunc list  done')          
        
end = time.perf_counter()
 
# find elapsed time in seconds
ms = (end-start) 
print(f"Elapsed {ms:.03f} secs.")        

DDIT4#887
CALM2#243
CTHRC1#681
LAPTM4A#673
MYL6#082
ANXA2#590
B2M#891
LGALS1#105
CD86#895
ACTB#594
COL1A1#499
CD99#029
SLITRK6#481
LHX8#469
COL6A1#428
IFITM3#203
ANXA5#782
ITM2B#732
CXCL14#484
SLIT3#813
MDK#035
SERPINF1#283
SFRP2#122
RARRES2#496
PTN#466
SFRP4#051
THY1#851
PDGFRB#273
RUNX2#231
PRRX1#695
JUNB#473
DPEP1#983
ALPL#783
ISG15#483
CTSK#450
FTL#788
MMP2#069
ACTA2#974
LUM#744
COL3A1#076
COL6A3#438
PCOLCE#465
COL1A2#403
MFAP2#417
MXRA8#422
BGN#905
FBLN1#994
ASPN#087
POSTN#809
TAGLN#547
DCN#893
IL11RA#773
WNT5A#990
BMPR1A#32
ACVRL1#950
TGFB3#747
MRFAP1#091
PRSS23#458
MYL9#058
TNFRSF14#354
RILP#366
NUPR1#230
CTGF#101
WISP2#792
ZFP106#781
ZNF474#907
(PRELP#223, shared)
(TNXA#093, shared)
(CLDN5#482, shared)
(LBP#461, shared)
(ANGPT1#939, shared)
(FBXO7#178, shared)
(FAM104A#319, shared)
(ACTA1#872, shared)
(B3GNT8#338, shared)
(LIF#266, shared)
(MSLN#885, shared)
(IL7#693, shared)
(GLI3#201, shared)
(ATF2#984, shared)
(OSMR#729, shared)
(FGF7#741, shared)
(BMPR1B#975, shared)
(PRRX2

In [12]:
## 2-APPLY transitive reduction algorithm on all lolo pathways (e.g. if a--lolo-->b and b--lolo-->c and a--lolo-->c, then we remove an edge shows a--lolo-->c)
remaining_edges = BU.transitive_reduction_part(res_sh_dys_fn,genes_cluster)
## 3- build network for the right side (a.k.a low-low pathways)
g_right = nx.DiGraph() #g_final   DELETE COM.
g_right = BU.build_net(g_right,remaining_edges,genes,genes_BU)


step3 done
step4 done


In [13]:
impo_genes=["MYL9",'FN1', 'ACVRL1', 'IHH', 'BMPR1A',"ACTA2","TAGLN"] # genes that we are interested in 

attrs={}
for s in impo_genes:
    for n in g_right.nodes():
        if s in n:
            print('found it',n)
            attrs[n]=g_right.nodes[n]

found it MYL9#058
found it ACVRL1#950
found it BMPR1A#32
found it ACTA2#974
found it TAGLN#547


In [14]:
attrs



{'MYL9#058': {'label': '(MYL9#058, dysfunc)', 'size': 5, 'color': 'skyblue'},
 'ACVRL1#950': {'label': '(ACVRL1#950, dysfunc)',
  'size': 5,
  'color': 'skyblue'},
 'BMPR1A#32': {'label': '(BMPR1A#32, dysfunc)', 'size': 5, 'color': 'skyblue'},
 'ACTA2#974': {'label': '(ACTA2#974, dysfunc)', 'size': 5, 'color': 'skyblue'},
 'TAGLN#547': {'label': '(TAGLN#547, dysfunc)', 'size': 5, 'color': 'skyblue'}}

In [15]:
# make the size of important genes bigger
attrs={
 'ACVRL1#950': {'label': '(ACVRL1#950, per)', 'size':25, 'color': 'skyblue'},
 'BMPR1A#32': {'label': '(BMPR1A#32, per)', 'size': 25, 'color': 'skyblue'},

'MYL9#058': {'label': '(MYL9#058, per)', 'size': 25, 'color': 'skyblue'},

 'ACTA2#974': {'label': 'ACTA2#974', 'size': 25, 'color': 'skyblue'},
 'TAGLN#547': {'label': 'TAGLN#547', 'size': 25, 'color': 'skyblue'}}
nx.set_node_attributes(g_right, attrs)

In [16]:
# right network shows all chains of lolo pathways
nt_right = Network("500px", "500px", heading="LOLO-pathways", notebook=True,cdn_resources='in_line')
nt_right.from_nx(g_right)
nt_right.show("BNET_lolo_right.html")

In [17]:
## 4- check all nodes from the low-low graph g_right that have at least one outgoing edge (i.e., their out-degree is not zero)
BU.g_right=g_right
outdeg = g_right.out_degree()
check_for_hilo_op = [n for n,v in set(outdeg) if v != 0]


connecting_points,BU.edge_write_p,g_right_plus_hilo=BU.add_single_hilo_op(check_for_hilo_op, genes)
connecting_points #potential genes from the right side network might connect right side to the leftside genes since we found a hilo for that



{'ACTA1#872',
 'ANGPT1#939',
 'B3GNT8#338',
 'BACE1#462',
 'BMPR1B#975',
 'C16orf77#046_CDH15#328_IBSP#370_SFRP5#468',
 'CCL19#072',
 'CDH5#677_KDR#934',
 'CLDN5#482',
 'CPEB1#578',
 'EGFR#983',
 'FAM104A#319',
 'FBXO7#178',
 'GLI2#537',
 'GLI3#201',
 'GRB2#049',
 'IGF2#409',
 'IL7#693',
 'ITSN1#776',
 'LBP#461',
 'LIF#266',
 'MSLN#885',
 'OSMR#729',
 'PRELP#223',
 'PRRX2#729',
 'RIPK3#139',
 'RUNX1#129',
 'SMO#29',
 'TNXA#093'}

In [18]:

nt_right_plus = Network("800px", "800px", heading="possible-sub_hilo+op", notebook=True)
nt_right_plus.from_nx(g_right_plus_hilo)
nt_right_plus.show("BNET_lolo_plus_hilo.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [19]:
## now, connect the right side network by adding hihi pathways.

# we can use the same procedures by finding all lolo pathways end with functional gene. a --hihi--> b == b--lolo--> a (The contrapositive Boolean rel for hihi is lolo)
BU.saved_gr=[]
BU.edge_write_F=set([])
BU.nodes_in_path_F=[]
BU.success=set([])
BU.nodes_in_path = list(set(BU.nodes_in_path))
for g in connecting_points:
    g1 = genes[g]
    g1.dfs('2', 'func', 1, [g1], [], lambda p, g: BU.hihi_path([], p,g, t_m='hilo'),'lolo')   # connect a hilo edge to the chain of lolo with only functional genes     
    
for gg in genes.keys():
    g1 = genes[gg]
    if g1.ct != 'func' and g1.ct!='dysfunc' and g1.name in  connecting_points: # start from shared, connect a hilo edge to the chain of lolo with  functional and shared genes, starts with a shared gene  
        g1.dfs('2', 'shared', 1, [g1], [], lambda p, g: BU.hihi_path([], p, g, t_m='hilo_shared'),'lolo')
        g1.dfs('2', 'func', 1, [g1], [], lambda p, g: BU.hihi_path([], p, g, t_m='hilo'),'lolo')

In [20]:
print(BU.success)

{'PRELP#223', 'CDH5#677_KDR#934', 'BACE1#462', 'RIPK3#139', 'PRRX2#729', 'DIP2C#504', 'BMPR1B#975', 'SMO#29', 'B3GNT8#338', 'GLI3#201', 'NOTCH3#238', 'C16orf77#046_CDH15#328_IBSP#370_SFRP5#468', 'EGFR#983', 'IGF1#541', 'RUNX1#129', 'HECTD2#568', 'CPEB1#578', 'ITSN1#776', 'FBXO7#178', 'GLI2#537', 'CLDN5#482', 'FAM104A#319', 'ACTA1#872', 'GRB2#049'}


In [21]:
BU.edge_write=BU.edge_write_F
BU.nodes_in_path_F=BU.nodes_in_path_F

remaining_edges_left = BU.transitive_reduction_part(res_sh_dys_fn,genes_cluster)

step3 done
step4 done


In [22]:
g_left = nx.DiGraph() 
g_left = BU.build_net(g_left,remaining_edges_left,genes,genes_BU)


In [23]:
BU.success=list(BU.success)
desc = nx.descendants(g_right_plus_hilo,  BU.success[0])
paths = [path for p in desc for path in nx.all_simple_paths(g_right_plus_hilo,  BU.success[0], p)]
paths
for s in BU.success[1:]:
    print(s)
    desc = nx.descendants(g_right_plus_hilo,  s)
    paths= paths +[path for p in desc for path in nx.all_simple_paths(g_right_plus_hilo,  s, p)]

net_sub_T = nx.DiGraph()
for p in paths:
    nx.add_path(net_sub_T, p)    

CDH5#677_KDR#934
BACE1#462
RIPK3#139
PRRX2#729
DIP2C#504
BMPR1B#975
SMO#29
B3GNT8#338
GLI3#201
NOTCH3#238
C16orf77#046_CDH15#328_IBSP#370_SFRP5#468
EGFR#983
IGF1#541
RUNX1#129
HECTD2#568
CPEB1#578
ITSN1#776
FBXO7#178
GLI2#537
CLDN5#482
FAM104A#319
ACTA1#872
GRB2#049


In [24]:

G = g_right_plus_hilo
H = net_sub_T # subnetwork with genes connecting the right_sub network to the functional network (left sub-network) 
R = G.copy()
R.remove_nodes_from(n for n in G if n not in H)
R.remove_edges_from(e for e in G.edges if e not in H.edges)

In [25]:
Boolean_net_final =  nx.compose(g_left,R) 

In [26]:
outdeg = Boolean_net_final.degree()
set(outdeg)
remove_hilo_alone = [n for n,v in set(outdeg) if v == 1]

In [27]:

color=nx.get_edge_attributes(Boolean_net_final,'color')
  
remove=[n for n in remove_hilo_alone if len(Boolean_net_final.edges(n))==1 and color[list(Boolean_net_final.edges(n))[0]]=='red']
remove

for i in remove:
    Boolean_net_final.remove_node(i)

In [28]:
attrs={}
for s in impo_genes:
    for n in Boolean_net_final.nodes():
        if s in n:
            print('yes',n)
            attrs[n]=Boolean_net_final.nodes[n]

yes MYL9#058
yes FN1#199
yes ACVRL1#950
yes IHH#358
yes BMPR1A#32
yes ACTA2#974
yes TAGLN#547


In [29]:
attrs={
 'ACVRL1#950': {'label': '(ACVRL1#950, per)', 'size': 25, 'color': 'skyblue'},
 'BMPR1A#32': {'label': '(BMPR1A#32, per)', 'size': 25, 'color': 'skyblue'},
    'MYL9#058': {'label': '(MYL9#058, per)', 'size': 25, 'color': 'skyblue'},
 'ACTA2#974': {'label': 'ACTA2#974', 'size': 25, 'color': 'skyblue'},
 'TAGLN#547': {'label': 'TAGLN#547', 'size': 25, 'color': 'skyblue'},
 'FN1#199': {'label': '(FN1#199, gr)', 'size': 25, 'color': 'lightgreen'},
 'IHH#358': {'label': '(IHH#358, gr)', 'size': 25, 'color': 'lightgreen'}}


In [30]:
nx.set_node_attributes(Boolean_net_final, attrs)
nt = Network("500px", "500px", heading="BooleaN net-final", notebook=True,cdn_resources='in_line')
nt.from_nx(Boolean_net_final)

nt.show("final_BNET.html")


In [31]:

color = nx.get_edge_attributes(Boolean_net_final, "color")
edge_write_F_bin=[]
for f,t in color:
    rc=color[(f,t)]
    r= "2" if rc=='navy' else "3" if rc=='green' else "4" if rc=='red' else "5" if  rc=='cyan' else '6'
    info=','.join([ f,t, r])
   
    edge_write_F_bin.append(info)

In [32]:
len(edge_write_F_bin)

551

In [33]:
len(Boolean_net_final)  # 139 genes+ 2 clustered nodes

141

In [34]:
# The genes selected for BTR
impo_genes2=["PHOSPHO1","ACRBP","FBXO7","BMPR1A","MYL9","ACTA2","ACRVL1","PDGFRB","CTSK","WISP2","IHH","FERMT1","SOX9","EGFR","TAGLN","ACVRL1","FN1"]
Boolean_net_final2 = Boolean_net_final.copy()
attrs=[]
for s in impo_genes2:
    for n in Boolean_net_final2.nodes():
        if s in n:
            print('found',n)
            attrs.append(n)
attrs

found PHOSPHO1#218
found ACRBP#717
found FBXO7#178
found BMPR1A#32
found MYL9#058
found ACTA2#974
found PDGFRB#273
found CTSK#450
found WISP2#792
found IHH#358
found FERMT1#796
found SOX9#935
found EGFR#983
found TAGLN#547
found ACVRL1#950
found FN1#199


['PHOSPHO1#218',
 'ACRBP#717',
 'FBXO7#178',
 'BMPR1A#32',
 'MYL9#058',
 'ACTA2#974',
 'PDGFRB#273',
 'CTSK#450',
 'WISP2#792',
 'IHH#358',
 'FERMT1#796',
 'SOX9#935',
 'EGFR#983',
 'TAGLN#547',
 'ACVRL1#950',
 'FN1#199']

In [35]:
G = Boolean_net_final2
R = G.copy()
R.remove_nodes_from(n for n in G if n not in attrs)


In [36]:
nt = Network("500px", "500px", heading="BooleaNet subnet", notebook=True,cdn_resources='in_line')
nt.from_nx(R)
nt.show("subnetwork_impogenes.html")

In [37]:
# Add some edges between important genes that are removed by transitive resuction algo.
cfile = "Global-net-3-0.15.rl"
net = bone.BINetwork(cfile)
net.init()
ids={
                'ACVRL1#950': '226950_at',
 'BMPR1A#32': '204832_s_at',
 'ACTA2#974': '200974_at',
 'TAGLN#547': '205547_s_at',
    'MYL9#058': '201058_s_at',
              }


lolo_edges=set([])
for k_i, i in ids.items(): # k_iname, i=probeid
    ru = bone.BIGraph.getNCodes(net, [i])

    for k_j, j in ids.items():
        rel=[k for k in set(ru.keys()) if j in ru[k].keys()]
        if len(rel)==1:
            if int(rel[0])==2:
                lolo_edges.add((k_i,k_j,rel[0]))
lolo_edges

{('ACTA2#974', 'ACVRL1#950', 2),
 ('BMPR1A#32', 'ACTA2#974', 2),
 ('BMPR1A#32', 'ACVRL1#950', 2),
 ('BMPR1A#32', 'MYL9#058', 2),
 ('BMPR1A#32', 'TAGLN#547', 2),
 ('TAGLN#547', 'ACVRL1#950', 2)}

In [38]:


contrapositive_edges=[("EGFR#983", "FBXO7#178",'4'),("ACRBP#717", "PHOSPHO1#218", '2'),("FBXO7#178","ACRBP#717",'2')] # 


new_edges=list(lolo_edges)+contrapositive_edges




for f, t, r in new_edges:

    r=str(r)
    if r=='2':

        R.add_edge(f, t, rel='2', color='navy',arrows={'to':{'enabled':True, 'type':'arrow'}})  
        R.add_node(f, label=genes[f].pretty(), size=5*(f.count("#")), color=genes[f].color)
        R.add_node(t, label=genes[t].pretty(), size=5*(t.count("#")),color=genes[t].color)


    elif r=='4':
        R.add_edge(f, t, rel='4', color='red')  
        R.add_node(f, label=genes[f].pretty(), size=5*(f.count("#")),color=genes[f].color)
        R.add_node(t, label=genes[t].pretty(), size=5*(t.count("#")),color=genes[t].color)    


In [39]:



nt = Network("500px", "500px", heading="Boolean_subnet", notebook=True,cdn_resources='in_line',directed=True)
nt.from_nx(R)


nt.show("final_subbin-w_contrapositive.html")
